In [1]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from config import *
from pprint import pprint

/home/azureuser/Ambarish/sentence-llama-index/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [3]:
client = QdrantClient(URL,port=6333)

In [4]:
vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)

In [7]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import SentenceSplitter

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [8]:
# base node parser is a sentence splitter
text_splitter = SentenceSplitter()

In [9]:
from llama_index.llms.azure_openai import AzureOpenAI
llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name=deployment_id_gpt4,
    api_key=key,
    azure_endpoint=endpoint,
    api_version=api_version,
)

In [10]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [11]:
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2",max_length=512
)

In [12]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

In [13]:
from llama_index.core  import VectorStoreIndex

In [14]:
sentence_index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [15]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

In [16]:
#query = "Describe about Tata Steel Kalinganagar?"
query = "What should the government do about road safety?"

In [17]:
response = query_engine.query(query)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
pprint(response.response)

('The government should make road safety a political priority, develop a '
 'multidisciplinary approach to road safety, appoint a lead agency for road '
 'safety, set appropriate road safety targets and establish national road '
 'safety plans to achieve them, support the creation of safety advocacy '
 'groups, create budgets for road safety, and increase investment in '
 'demonstrably effective road safety activities. Road safety is a shared '
 'responsibility of the government and a range of civil society stakeholders, '
 'and the success of road safety strategies in all countries depends upon a '
 'broad base of support and common action from all stakeholders.')


In [23]:
query = "What is  ENERGY INTENSITY AT TSJ?"
query = "Describe the Highway police job"

In [24]:
query

'Describe the Highway police job'

In [25]:
response = query_engine.query(query)

In [27]:
pprint(response.response)

('The need for a dedicated highway police force to enforce road safety on '
 'National Highways was discussed by a committee. The committee recognized '
 'that any move to create a dedicated highway police force could be resisted '
 'by the State governments. However, the committee felt that it was necessary '
 'to have a dedicated force capable of policing the National Highways using '
 'modern technology and equipment. The government was examining the '
 'possibility of employing the Central Industrial Security Force (CISF) for '
 'this purpose. The context does not provide a detailed description of the '
 'highway police job.')
